<a href="https://colab.research.google.com/github/GreenStreetQuant/QQQ---Tweet/blob/master/qqq_bearish_or_bullish_tweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yfinance

In [ ]:
pip install feature_selector

In [5]:
import tweepy
import pandas as pd 
import datetime as dt 
import yfinance as yf
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from feature_selector import FeatureSelector

In [59]:
consumer_key = 'asdf'
consumer_secret = 'asdf'
access_token = 'asdfasdf'
access_token_secret = 'asdf'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [3]:
qqq = yf.download("QQQ",period='2y')
spy = yf.download("SPY",period='2y')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [31]:
def exp_smooth_data(data):

  smooth_list = []

  #first value y for s 
  smooth_list.append(data.iloc[0])

  for v in data[1:]:
    st = 0.2 * v + (1 - 0.2) * smooth_list[-1]
    smooth_list.append(st)

  return smooth_list 

def computeRSI(data):
    diff = data.diff(1).dropna()        # diff in one field(one day)

    #this preservers dimensions off diff values
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change is equal to the positive difference, otherwise equal to zero
    up_chg[diff > 0] = diff[ diff > 0]
    
    # down change is equal to negative deifference, otherwise equal to zero
    down_chg[diff < 0] = diff[ diff < 0 ]
    
    # check pandas documentation for ewm
    # we set com=time_window-1 so we get decay alpha=1/time_window

    up_chg_avg   = up_chg.ewm(com=14-1 , min_periods=14).mean()
    down_chg_avg = down_chg.ewm(com=14-1 , min_periods=14).mean()
    
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi

def get_features(qqq,spy):

  df_smoothed = pd.DataFrame()

  close_smooth = exp_smooth_data(qqq['Close'])

  df_smoothed['close_smooth'] = close_smooth

  open_smooth = exp_smooth_data(qqq['Open'])

  df_smoothed['Open_smooth'] = open_smooth

  high_smooth = exp_smooth_data(qqq['High'])

  df_smoothed['high_smooth'] = high_smooth

  low_smooth = exp_smooth_data(qqq['Low'])

  df_smoothed['low_smooth'] = low_smooth

  volume_smooth = exp_smooth_data(qqq['Volume'])

  df_smoothed['volume_smooth'] = volume_smooth

  df_sp_smoothed = pd.DataFrame()

  close_smooth_sp = exp_smooth_data(spy['Close'])

  df_sp_smoothed['close_smooth'] = close_smooth_sp

  df_smoothed['rsi'] = computeRSI(df_smoothed['close_smooth'])
  df_smoothed['william'] = (df_smoothed['high_smooth'].rolling(14).max() - df_smoothed['close_smooth'])/(df_smoothed['high_smooth'].rolling(14).max() - df_smoothed['low_smooth'].rolling(14).min()) * -100
  df_smoothed['stch_osc'] = 100 * (df_smoothed['close_smooth'] - df_smoothed['low_smooth'].rolling(14).min())/(df_smoothed['high_smooth'].rolling(14).max() - df_smoothed['low_smooth'].rolling(14).min())
  df_smoothed['price_rate_change'] = (df_smoothed['close_smooth'] - df_smoothed['close_smooth'].shift(14))/df_smoothed['close_smooth'].shift(14)
  df_smoothed['log_price'] = np.log(df_smoothed['close_smooth'])
  df_smoothed['log_mov'] = df_smoothed['log_price'].rolling(6).mean()
  df_smoothed['log_diff'] = df_smoothed['log_price'] - df_smoothed['log_mov']
  df_smoothed['fast_mov'] = df_smoothed['close_smooth'].rolling(3).mean()
  df_smoothed['slow_mov'] = df_smoothed['close_smooth'].rolling(7).mean()
  df_smoothed['mov_diff'] = df_smoothed['fast_mov'] - df_smoothed['slow_mov']
  df_smoothed['mac_fast'] = df_smoothed['close_smooth'].rolling(7).mean()
  df_smoothed['mac_slow'] = df_smoothed['close_smooth'].rolling(20).mean()
  df_smoothed['mac_diff'] = df_smoothed['mac_fast'] - df_smoothed['mac_slow']
  df_smoothed['volume_log'] = np.log(df_smoothed['volume_smooth'])
  df_smoothed['pct_change'] = df_smoothed['close_smooth'].pct_change()
  df_smoothed['z_score'] = (df_smoothed['close_smooth'] - df_smoothed['close_smooth'].rolling(7).mean())/df_smoothed['close_smooth'].std()
  df_smoothed['sp_return'] = df_sp_smoothed['close_smooth'].pct_change(14)
  df_smoothed['return_two_week'] = df_smoothed['close_smooth'].pct_change(14)
  df_smoothed['return_day'] = df_smoothed['close_smooth'].pct_change(1)
  df_smoothed['return_month'] = df_smoothed['close_smooth'].pct_change(5)
  df_smoothed['return_two_day'] = df_smoothed['close_smooth'].pct_change(2)
  df_smoothed['return_week'] = df_smoothed['close_smooth'].pct_change(5)
  df_smoothed['return_diff_sp'] = df_smoothed['return_day'] - df_smoothed['sp_return']
  df_smoothed['return_sp_std'] = df_smoothed['sp_return'].rolling(14).std()
  df_smoothed['return_std'] = df_smoothed['return_week'].rolling(14).std()
  df_smoothed['last_close'] = df_smoothed['close_smooth'].shift(14)
  df_smoothed['last_open'] = df_smoothed['Open_smooth'].shift(14)
  df_smoothed['last_high'] = df_smoothed['high_smooth'].shift(14)
  df_smoothed['last_low'] = df_smoothed['low_smooth'].shift(14)
  df_smoothed['high_low'] = df_smoothed['last_high'] - df_smoothed['last_low']
  df_smoothed.head()

  df_smoothed = df_smoothed.dropna()

  df_complete = df_smoothed

  df_complete['win'] = np.where((df_complete['close_smooth'].shift(-5) > df_complete['close_smooth']), 1, 0)
  return df_complete

def get_bearish_or_bullish_signal(df_complete):
  train = df_complete.drop(columns=['win'])
  train_labels = df_complete['win']
  fs = FeatureSelector(data=train, labels=train_labels)
  fs.identify_collinear(correlation_threshold=0.975)
  fs.identify_zero_importance(task='regression',eval_metric='auc',n_iterations=10,early_stopping=True)
  fs.identify_low_importance(cumulative_importance = 0.99)
  all_to_remove = fs.check_removal()
  clean_removed_db = train.drop(columns = all_to_remove)
  split = len(clean_removed_db)-1
  X_train, X_test = clean_removed_db, clean_removed_db[split:]
  y_train = train_labels
  classifier = RandomForestClassifier(n_estimators=3,max_depth=30,max_features='sqrt',min_samples_leaf=27,min_samples_split=2,random_state=42)
  classifier.fit(X_train,y_train)
  y_pred = classifier.predict(X_test)
  return y_pred

In [48]:
def publictweet():
  data = get_features(qqq,spy)
  signal = get_bearish_or_bullish_signal(data)

  prediction = y_pred.tolist()

  if dt.date.today().weekday() == 0:
    if prediction == 1:
      tweettopublish = 'The week prediction for QQQ is Bullish. Last Close: ${:.2f}'.format(qqq['Close'].iloc[-1])
    else:
      tweettopublish = 'The week prediction for QQQ is Bearish. Last Close: ${:.2f}'.format(qqq['Close'].iloc[-1])

  api.update_status(tweettopublish)